In [ ]:
# @title 💻 1.1 Check CUDA and Python Version (tested with CUDA 12.2 + Python 3.10.12)

# This notebook is supposed to be connected to NVIDIA drivers with CUDA.
# If this doesn't load check that GPU is selected as hardware accelerator under Edit -> Notebook settings.
!nvidia-smi
!python -V

Wed Mar  5 05:26:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# @title 1.2 👆 Select Checkpoint
!pip install ipywidgets==7.7.1 --quiet

from IPython.display import display
import ipywidgets as widgets
import shutil

ckpt_dic = {
    "Original (higher quality)": "prs-eth/marigold-v1-0",
    "LCM (faster)": "prs-eth/marigold-lcm-v1-0",
}

ckpt_name = 'LCM (faster)'
ckpt_path = ckpt_dic[ckpt_name]
w = widgets.Dropdown(
    options=['Original (higher quality)', 'LCM (faster)'],
    value=ckpt_name,
    description='Checkpoint:',
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        ckpt_name = change['new']
        ckpt_path = ckpt_dic[ckpt_name]
        # print(f"Change to checkpoint: {ckpt_name} ({ckpt_path})")

w.observe(on_change)

display(w)
# print(f"{ckpt_path = }")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.0 MB/s eta 0:00:00


Dropdown(description='Checkpoint:', index=1, options=('Original (higher quality)', 'LCM (faster)'), value='LCM…

## 1.3 🏗️ Installation and Initialization

We provide two ways to run the pipeline. You can choose either ONE to run:

### 🅱 1.3-A Using code from the [full repository](https://github.com/prs-eth/marigold)

In [ ]:
# @title 📦 i. Clone repository

%%shell
cd /content

if [ -d "Marigold" ]; then
    cd Marigold
    git pull
else
    git clone https://github.com/prs-eth/Marigold.git
    cd Marigold
fi


Cloning into 'Marigold'...
remote: Enumerating objects: 633, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 633 (delta 215), reused 153 (delta 153), pack-reused 370 (from 2)
Receiving objects: 100% (633/633), 5.70 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (368/368), done.


In [ ]:
# @title 🔨 ii. Install dependencies (might take minutes)
%%shell

cd /content/Marigold

# pip install -r requirements.txt --upgrade  --quiet
pip install accelerate diffusers matplotlib scipy torch transformers --quiet

# for progress bar
pip install ipywidgets==7.7.1 --quiet

pip install -r requirements+.txt -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.7 MB/s eta 0:00:00
 

In [ ]:
# @title 🔧 iii. Setup directories

import os

# Directories
repo_dir = "/content/Marigold"
input_dir = os.path.join(repo_dir, "input")
output_dir = os.path.join(repo_dir, "output")
output_dir_color = os.path.join(output_dir, "depth_colored")
output_dir_tif = os.path.join(output_dir, "depth_bw")
output_dir_npy = os.path.join(output_dir, "depth_npy")

os.chdir(repo_dir)



In [ ]:
# @title 🏇 iv. Initialize pipeline

# change model cache directory
# !export HF_HOME=/content/Marigold/checkpoint

from marigold import MarigoldPipeline

pipe = MarigoldPipeline.from_pretrained(ckpt_path)

pipe = pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

tokenizer%2Fvocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Path to the zip file in your Google Drive
zip_file_path = '/content/drive/My Drive/datasets/hypersim_processed.zip'

# Destination folder where you want to unzip the files
unzip_path = '/content/hypersim_processed/'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Now you can load the dataset from the unzipped directory
import os
print(os.listdir(unzip_path))


['hypersim_processed']


In [ ]:
%%shell

mv /content/hypersim_processed/hypersim_processed/val/filename_list_val.txt /content/Marigold/data_split/hypersim/filename_list_val_filtered.txt

In [ ]:
%%shell

export BASE_DATA_DIR="/content/hypersim_processed"

bash script/eval/61_infer_hypersim.sh

+ ckpt=prs-eth/marigold-v1-0
+ subfolder=eval
+ python infer.py --checkpoint prs-eth/marigold-v1-0 --seed 1234 --base_data_dir /content/hypersim_processed --denoise_steps 50 --ensemble_size 5 --processing_res 512 --dataset_config config/dataset/data_hypersim_val.yaml --output_dir output/eval/hypersim_val/prediction
2025-03-05 07:08:13.622256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741158493.653347   26921 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741158493.665062   26921 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
arguments: Namespace(checkpoint='prs-eth/marigold-v1-0', dataset_config='config/dataset/data_hypersim_val.yaml', base_da

In [ ]:
%%shell
export BASE_DATA_DIR="/content/hypersim_processed"

bash script/eval/62_eval_hypersim.sh

+ subfolder=eval
+ python eval.py --base_data_dir /content/hypersim_processed --dataset_config config/dataset/data_hypersim_val.yaml --alignment least_square --prediction_dir output/eval/hypersim_val/prediction --output_dir output/eval/hypersim_val/eval_metric
Evaluating: 100% 200/200 [00:24<00:00,  8.04it/s]


In [ ]:
!zip -r hypersim_val50.zip /content/Marigold/output/eval/hypersim_val


  adding: content/Marigold/output/eval/hypersim_val/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0014.npy (deflated 13%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0048.npy (deflated 16%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0034.npy (deflated 14%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0091.npy (deflated 12%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0054.npy (deflated 14%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0039.npy (deflated 12%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0097.npy (de

In [ ]:
!dir


config		 doc			   infer.py	README.md	    run.py
CONTRIBUTING.md  environment.yaml	   LICENSE.txt	requirements++.txt  script
data_split	 eval.py		   marigold	requirements+.txt   src
dir.zip		 hypersim_val_metrics.zip  output	requirements.txt    train.py


In [ ]:
!cp hypersim_val50.zip /content/drive/MyDrive/


In [ ]:
!zip -r hypersim_val.zip /content/Marigold/output/eval/hypersim_val/

from google.colab import files
files.download("/content/Marigold/hypersim_val.zip")

updating: content/Marigold/output/eval/hypersim_val/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0014.npy (deflated 13%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0048.npy (deflated 16%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0034.npy (deflated 14%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0091.npy (deflated 13%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0054.npy (deflated 14%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0039.npy (deflated 12%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0097.npy (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("hello")

hello
